In [8]:
import mediapipe as mp
import cv2
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 

2025-11-08 15:01:57.328459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. This error is frequently encountered on macOS when running an x86 Python installation on ARM hardware. In this case, try installing an ARM build of Python. Otherwise, you may be able work around this issue by building jaxlib from source.

In [ ]:
DATA_DIR = '../data/asl_alphabet_train/asl_alphabet_train'
OUTPUT_FILE = '../data/asl_landmarks_train.csv'